In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
exec(open("plot_utils.py").read())

/Users/adsurbum/anaconda2/envs/py371/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [95]:
country_name = 'Denmark'
file_name = '../output/2020-05-12-fixed-deltat0-new-normal-100k/inference/{}.npz'.format(country_name)
data = np.load(file_name)



In [96]:
var_names = list(data['var_names'])
incidences = data['incidences']
chain = data['chain']
nsteps, ndim, N, Td1, Td2, τ_model = data['params']
nburn = int(nsteps * 0.6)
sample = chain[:, nburn:, :].reshape(-1, ndim)

medians = [np.median(sample[:,i]) for i in range(len(var_names))]
means = [sample[:,i].mean() for i in range(len(var_names))]
lnprobability = data['lnprobability'][:, nburn:]
MAP = sample[lnprobability.argmax()]

In [97]:
list(zip(var_names, means, medians))

[('Z', 3.7760482200115915, 3.8831139025358827),
 ('D', 3.4661229705973677, 3.446953886769261),
 ('μ', 0.39694258261800125, 0.3494410585274323),
 ('β', 1.0539407076749148, 1.0153880884790263),
 ('α1', 0.0927636889106337, 0.04447129657541335),
 ('λ', 0.3670355003296399, 0.313196703327396),
 ('α2', 0.5581233315698805, 0.5609320861881292),
 ('E0', 782.8967506034869, 532.4624493355763),
 ('Iu0', 950.6886963794642, 749.8657321073466),
 ('Δt0', 2.877351056793107, 2.63310383706265),
 ('τ', 16.295854877458915, 15.860176607721545)]

## medians

In [100]:
seed_max = 3000
params_bounds = {
    'Z' : (2, 5),
    'D' : (2, 5),
    'μ' : (0.2, 1),
    'β' : (0.8, 1.5),
    'α1' : (0.02, 1),
    'λ'  : (0, 1),
    'α2' : (0.02, 1),
    'E0' : (0, seed_max),
    'Iu0' : (0, seed_max),
    'Δt0' : (1,5) #how much zeros before the first incident
}

## loglikelihood code:
θ = medians
X = incidences
Z, D, μ, β, α1, λ, α2, E0, Iu0, Δt0, τ = θ
τ = int(τ) # for explanation see https://github.com/dfm/emcee/issues/150
Δt0 = int(Δt0)

total_zeros = params_bounds['Δt0'][1]
unrellevant_zeros = total_zeros - Δt0
τ = τ - unrellevant_zeros
X = X[unrellevant_zeros:]
ndays = len(X)

S, E, Ir, Iu, R, Y = simulate(Z, D, μ, β, α1, λ, α2, E0, Iu0, Δt0, τ, ndays, N)
p1 = 1/Td1
p2 = 1/Td2
Xsum = X.cumsum() 
n = Y[1:] - Xsum[:-1] 
n = np.maximum(0, n)
p = ([p1] * τ + [p2] * (ndays - τ))[1:]

loglik = scipy.stats.poisson.logpmf(X[Δt0:], n[Δt0-1:] * p[Δt0-1:])

In [105]:
def r(arr):
    return [str(int(a)) if not np.isinf(a) else '-inf' for a in arr]
def r_arrays(arr):
    return [r(a) for a in arr]

loglik_to_print = np.array([0]*Δt0+list(loglik))
print('\t'.join(['day', 'loglik', 'X', 'Xsum', 'Y', 'n']))
arr = r_arrays(list(zip(range(len(X)),loglik_to_print,X, Xsum, Y, n)))
for a in arr:
    print('\t'.join(a))

day	loglik	X	Xsum	Y	n
0	0	0	0	0	6
1	0	0	0	6	14
2	-1	1	1	14	21
3	-2	0	1	22	30
4	-2	1	2	31	39
5	-2	1	3	41	48
6	-3	1	4	51	57
7	-4	1	5	61	67
8	-3	3	8	72	76
9	-4	2	10	84	86
10	-2	10	20	96	88
11	-4	3	23	108	85
12	-3	8	31	108	225
13	-20	7	38	256	349
14	-5	75	113	387	393
15	-43	151	264	506	353
16	-176	252	516	617	206
17	-123	160	676	722	146
18	-112	128	804	822	114
19	-2	23	827	918	183
20	-7	48	875	1010	224
21	-7	57	932	1099	253
22	-24	92	1024	1185	245
23	-25	91	1115	1269	234
24	-2	36	1151	1349	277
25	-29	104	1255	1428	248
26	-11	71	1326	1503	251
27	-10	69	1395	1577	253
28	-8	65	1460	1648	257
29	-61	131	1591	1717	192
30	-91	133	1724	1783	124
31	-176	153	1877	1848	34


## means

In [108]:
## loglikelihood code:
θ = means
X = incidences
Z, D, μ, β, α1, λ, α2, E0, Iu0, Δt0, τ = θ
τ = int(τ) # for explanation see https://github.com/dfm/emcee/issues/150
Δt0 = int(Δt0)

total_zeros = params_bounds['Δt0'][1]
unrellevant_zeros = total_zeros - Δt0
τ = τ - unrellevant_zeros
X = X[unrellevant_zeros:]
ndays = len(X)

S, E, Ir, Iu, R, Y = simulate(Z, D, μ, β, α1, λ, α2, E0, Iu0, Δt0, τ, ndays, N)
p1 = 1/Td1
p2 = 1/Td2
Xsum = X.cumsum() 
n = Y[1:] - Xsum[:-1] 
n = np.maximum(0, n)
p = ([p1] * τ + [p2] * (ndays - τ))[1:]

loglik = scipy.stats.poisson.logpmf(X[Δt0:], n[Δt0-1:] * p[Δt0-1:])

In [109]:
def r(arr):
    return [str(int(a)) if not np.isinf(a) else '-inf' for a in arr]
def r_arrays(arr):
    return [r(a) for a in arr]

loglik_to_print = np.array([0]*Δt0+list(loglik))
print('\t'.join(['day', 'loglik', 'X', 'Xsum', 'Y', 'n']))
arr = r_arrays(list(zip(range(len(X)),loglik_to_print,X, Xsum, Y, n)))
for a in arr:
    print('\t'.join(a))

day	loglik	X	Xsum	Y	n
0	0	0	0	0	21
1	0	0	0	21	46
2	-3	1	1	46	73
3	-8	0	1	74	103
4	-9	1	2	104	136
5	-12	1	3	138	171
6	-16	1	4	174	210
7	-20	1	5	214	252
8	-19	3	8	257	295
9	-26	2	10	303	343
10	-16	10	20	353	387
11	-33	3	23	407	443
12	-28	8	31	466	435
13	-51	7	38	466	773
14	-16	75	113	811	1014
15	-4	151	264	1127	1160
16	-11	252	516	1424	1195
17	-7	160	676	1711	1315
18	-25	128	804	1991	1464
19	-169	23	827	2268	1715
20	-155	48	875	2542	1939
21	-170	57	932	2814	2153
22	-144	92	1024	3085	2331
23	-168	91	1115	3355	2508
24	-296	36	1151	3623	2741
25	-202	104	1255	3892	2904
26	-279	71	1326	4159	3099
27	-311	69	1395	4425	3296
28	-348	65	1460	4691	3496
29	-259	131	1591	4956	3629
30	-273	133	1724	5220	3760
31	-261	153	1877	5484	3870
